## Ques 1

In [6]:
!apt-get install -y glpk-utils
!pip install pyomo
!pip install nashpy

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
glpk-utils is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [14]:
import numpy as np
import nashpy as nash
A = np.array([[2, 4, 0],
              [1, 1, 4],
              [0, 8, 7]])
print("Payoff matrix A:")
print(A)
game = nash.Game(A, -A)
equilibria = list(game.support_enumeration())
for i in equilibria:
    print("\nEquilibrium found:")
    print("Row player's strategy:", i[0])
    print("Column player's strategy:", i[1])
    value = i[0].dot(A).dot(i[1])
    print("Value of the game (for the row player):", value)

Payoff matrix A:
[[2 4 0]
 [1 1 4]
 [0 8 7]]

Equilibrium found:
Row player's strategy: [0.6 0.4 0. ]
Column player's strategy: [ 0.8 -0.   0.2]
Value of the game (for the row player): 1.6000000000000003


In [15]:
from pyomo.environ import *

def solve_zero_sum_game_pyomo(A):
    n, m = A.shape
    model_r = ConcreteModel()
    model_r.I = RangeSet(0, n-1)
    model_r.x = Var(model_r.I, within=NonNegativeReals)
    model_r.z = Var(within=Reals)
    def obj_r(model):
        return model_r.z
    model_r.obj = Objective(rule=obj_r, sense=maximize)
    model_r.J = RangeSet(0, m-1)
    def saddle_r(model, j):
        return model_r.z - sum(A[i,j]*model_r.x[i] for i in model_r.I) <= 0
    model_r.saddle = Constraint(model_r.J, rule=saddle_r)
    def sum_x(model):
        return sum(model_r.x[i] for i in model_r.I) == 1
    model_r.sum_x = Constraint(rule=sum_x)
    SolverFactory('glpk').solve(model_r)
    x_opt = np.array([model_r.x[i].value for i in model_r.I])
    z_opt = model_r.z.value
    model_c = ConcreteModel()
    model_c.J = RangeSet(0, m-1)
    model_c.y = Var(model_c.J, within=NonNegativeReals)
    model_c.w = Var(within=Reals)


    def obj_c(model):
        return model_c.w
    model_c.obj = Objective(rule=obj_c, sense=minimize)
    model_c.I = RangeSet(0, n-1)
    def saddle_c(model, i):
        return model_c.w - sum(A[i,j]*model_c.y[j] for j in model_c.J) >= 0
    model_c.saddle = Constraint(model_c.I, rule=saddle_c)
    def sum_y(model):
        return sum(model_c.y[j] for j in model_c.J) == 1
    model_c.sum_y = Constraint(rule=sum_y)
    SolverFactory('glpk').solve(model_c)
    y_opt = np.array([model_c.y[j].value for j in model_c.J])
    w_opt = model_c.w.value
    return x_opt, z_opt, y_opt, w_opt




A = np.array([[2, 4, 0],
              [1, 1, 4],
              [0, 8, 7]])

x_opt, z_opt, y_opt, w_opt = solve_zero_sum_game_pyomo(A)

print("Optimal mixed strategy for the row player:", x_opt)
print("Value of the game (for the row player):", z_opt)
print()
print("Optimal mixed strategy for the col player:", y_opt)
print("Value of the game (for the col player):", w_opt)



Optimal mixed strategy for the row player: [0.6 0.4 0. ]
Value of the game (for the row player): 1.6

Optimal mixed strategy for the col player: [0.8 0.  0.2]
Value of the game (for the col player): 1.6


## Ques 2

In [13]:
import pandas as pd

def fictitious_play(A, T=10000):
    n, m = A.shape
    row_counts = np.zeros(n)
    col_counts = np.zeros(m)
    row_action = np.random.choice(n)
    col_action = np.random.choice(m)
    row_counts[row_action] += 1
    col_counts[col_action] += 1
    records = []

    for t in range(1, T + 1):
        row_dist = row_counts / np.sum(row_counts)
        col_dist = col_counts / np.sum(col_counts)
        row_payoffs = A.dot(col_dist)
        col_payoffs = -A.T.dot(row_dist)
        row_action = np.argmax(row_payoffs)
        col_action = np.argmax(col_payoffs)
        row_counts[row_action] += 1
        col_counts[col_action] += 1

        if t <= 50 or t > T - 50:
            records.append((t, row_action, col_action, row_dist.copy(), col_dist.copy()))

    final_row_dist = row_counts / np.sum(row_counts)
    final_col_dist = col_counts / np.sum(col_counts)
    value = final_row_dist.dot(A).dot(final_col_dist)

    return records, final_row_dist, final_col_dist, value



A = np.array([[2, 4, 0],
              [1, 1, 4],
              [0, 8, 7]])

table, row_strategy, col_strategy, val = fictitious_play(A, T=10000)

df = pd.DataFrame(
    table,
    columns=["Iteration", "RowAction BR", "ColAction BR", "RowDistribution", "ColDistribution"]
)
df["RowDistribution"] = df["RowDistribution"].apply(lambda arr: np.array2string(np.round(arr, 3), separator=','))
df["ColDistribution"] = df["ColDistribution"].apply(lambda arr: np.array2string(np.round(arr, 3), separator=','))

print("Payoff matrix A:")
print(A, "\n")

print("=== Final Results ===")
print("Estimated Row Strategy:", np.round(row_strategy, 3))
print("Estimated Column Strategy:", np.round(col_strategy, 3))
print("Estimated Value of the Game:", round(val, 3), "\n")

print("=== Fictitious Play Iteration Records  ===\n")
print(df.to_string(index=False))

Payoff matrix A:
[[2 4 0]
 [1 1 4]
 [0 8 7]] 

=== Final Results ===
Estimated Row Strategy: [0.593 0.404 0.003]
Estimated Column Strategy: [0.798 0.    0.202]
Estimated Value of the Game: 1.6 

=== Fictitious Play Iteration Records  ===

 Iteration  RowAction BR  ColAction BR     RowDistribution     ColDistribution
         1             2             0          [0.,1.,0.]          [0.,1.,0.]
         2             2             0       [0. ,0.5,0.5]       [0.5,0.5,0. ]
         3             0             0 [0.   ,0.333,0.667] [0.667,0.333,0.   ]
         4             0             0    [0.25,0.25,0.5 ]    [0.75,0.25,0.  ]
         5             0             0       [0.4,0.2,0.4]       [0.8,0.2,0. ]
         6             0             0 [0.5  ,0.167,0.333] [0.833,0.167,0.   ]
         7             0             0 [0.571,0.143,0.286] [0.857,0.143,0.   ]
         8             0             0 [0.625,0.125,0.25 ] [0.875,0.125,0.   ]
         9             0             0 [0.667,0.11